#FiniteStateMachine.jl

This notebook was created by R. Davis Born using Julia 0.3.11 in order to demonstrate the functionality of the FiniteStateMachine Julia package created by Jack Peterson.

FiniteStateMachine is the only package used in this notebook.

In [1]:
Pkg.add("FiniteStateMachine")

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of FiniteStateMachine
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [2]:
using FiniteStateMachine

##Basic Functionality

This package serves to create a simple finite state machine that can support any number of states and actions.

Actions must have at least one "from" state, exactly one "to" state, and exactly one "initial" state.

In [3]:
Ferry = state_machine({
    "initial" => "EastBank", # the initial state is where the machine starts when defined
    "events" => [{
            "name" => "CrossRiver", # the name of the action or "event"
            "from" => "EastBank", # the state the action can be taken from
            "to" => "WestBank", # the state the action leads to
            }, {
            "name" => "CrossRiver",
            "from" => "WestBank",
            "to" => "EastBank",
            }]
    })
;

The state can be changed by imposing an action (or "event") with fire(). See the current state with state_maching.current

In [4]:
@show Ferry.current
fire(Ferry,"CrossRiver")
@show Ferry.current
;

Ferry.current => "EastBank"
Ferry.current => "WestBank"


###Multiple "from" states and "final" states

State machines support multiple "from" states, and you have the option to define a "final" state that will throw a flag indicating that the state machine has "finished". Check the status of this flag with fire(state_machine, "finished"). Note that it is possible to define an action that can move away from the final state and flip the flag back to false.

In [5]:
Ferry = state_machine({
    "initial" => "Dock",
    "final" => "RiverBottom", # the final state indicates the state machine has "finished"
    "events" => [{
            "name" => "Launch",
            "from" => "Dock",
            "to" => "EastBank",
            }, {
            "name" => "CrossRiver",
            "from" => "EastBank",
            "to" => "WestBank",
            }, {
            "name" => "CrossRiver",
            "from" => "WestBank",
            "to" => "EastBank",
            }, {
            "name" => "Sink",
            "from" => ["EastBank", "WestBank"],
            "to" => "RiverBottom",
            }]
    })
;

In [6]:
@show Ferry.current # Ferry starts at the Dock
fire(Ferry,"Launch")
@show Ferry.current # Ferry launches to the East Bank
@show fire(Ferry, "finished") # Ferry is at East Bank, which is not the final state
fire(Ferry,"Sink")
@show Ferry.current # Ferry sunk to the Bottom of the River
@show fire(Ferry, "finished") # Ferry is at the Bottom of the River, which is the final state
;

Ferry.current => "Dock"
Ferry.current => "EastBank"
fire(Ferry,"finished") => false
Ferry.current => "RiverBottom"
fire(Ferry,"finished") => true


##A Game for You...

Enjoy this simple Monty Hall simulation implemented with FiniteStateMachine :)

In [5]:
MontyHall = state_machine({
    "initial" => "begin",
    "final" => "end",
    "events" => [{
            "name" => "GuessRight",
            "from" => ["begin","wrong"],
            "to" => "right",
            }, {
            "name" => "GuessWrong",
            "from" => ["begin","right"],
            "to" => "wrong",
            }, {
            "name" => "switch",
            "from" => "right",
            "to" => "lose",
            },  {
            "name" => "switch",
            "from" => "wrong",
            "to" => "win",
            }, {
            "name" => "stay",
            "from" => "right",
            "to" => "win",
            },  {
            "name" => "stay",
            "from" => "wrong",
            "to" => "lose",
            }, {
            "name" => "start",
            "from" => ["right", "wrong", "win", "lose", "end", "begin"],
            "to" => "begin",
            }, {
            "name" => "yes",
            "from" => ["win", "lose"],
            "to" => "begin",
            }, {
            "name" => "no",
            "from" => ["win", "lose"],
            "to" => "end",
            }]
    })

function MontyHallSetup()
    winner = int(ceil(3*rand()))
    doors = Array(String,3)
    for i = 1:length(doors)
        doors[i] = "GOAT"
    end
    doors[winner] = "CAR"
    return doors, winner
end
;

In [6]:
fire(MontyHall, "start")

println("Welcome to the Monty Hall Game Show!")
println("We have three doors for you to choose from. Two doors hide goats, and one door hides a NEW CAR!!")
println("You will guess which door hides the car, and then one of the other two doors will be shown to hide a goat...")
println("You will then be given the opportunity to switch your guess to the other unopened door...")
println("If your final guess is correct, you win the car!!")

while !fire(MontyHall, "finished")
    if MontyHall.current == "begin"
        (doors, winner) = MontyHallSetup()
        println()
        println("Go ahead and make your first guess! Type 1, 2, or 3:")
        guess = readline(STDIN)
        if guess == string(winner)
            fire(MontyHall, "GuessRight")
        else
            fire(MontyHall, "GuessWrong")
        end
    elseif MontyHall.current == "right" || MontyHall.current == "wrong"
        reveal = int(ceil(3*rand()))
        while reveal == winner || string(reveal) == guess
            reveal = int(ceil(3*rand()))
        end
        println()
        println("We will now reveal one of the losing doors...")
        println("Door ", reveal, " hides a GOAT! Would you like to switch or stay?")
        fire(MontyHall, readline(STDIN))
    elseif MontyHall.current == "win"
        println()
        println("Congratulations! You win! Door ", winner, " hides the car!")
        println("Pick up your keys on your way out. Would you like to play again (yes/no)?")
        fire(MontyHall, readline(STDIN))
    elseif MontyHall.current == "lose"
        println()
        println("Sorry! You lose! Door ", winner, " hid the car.")
        println("Enjoy your new pet goat. Would you like to play again (yes/no)?")
        fire(MontyHall, readline(STDIN))
    end
end

println()
println("Thanks for playing! Goodbye.")

Welcome to the Monty Hall Game Show!
We have three doors for you to choose from. Two doors hide goats, and one door hides a NEW CAR!!
You will guess which door hides the car, and then one of the other two doors will be shown to hide a goat...
You will then be given the opportunity to switch your guess to the other unopened door...
If your final guess is correct, you win the car!!

Go ahead and make your first guess! Type 1, 2, or 3:
STDIN> 1

We will now reveal one of the losing doors...
Door 2 hides a GOAT! Would you like to switch or stay?
STDIN> switch

Sorry! You lose! Door 1 hid the car.
Enjoy your new pet goat. Would you like to play again (yes/no)?
STDIN> yes

Go ahead and make your first guess! Type 1, 2, or 3:
STDIN> 2

We will now reveal one of the losing doors...
Door 1 hides a GOAT! Would you like to switch or stay?
STDIN> stay

Congratulations! You win! Door 2 hides the car!
Pick up your keys on your way out. Would you like to play again (yes/no)?
STDIN> no

Thanks for pla